# MASINÕPPE MUDELITE LOOMISE PROTSESS

#CRISP-DM standard



## 1) Äriliste eesmärkide seadmine 
Projekti eesmärkide püstitamine nii valdkonnaspetsiifiliste terminite kaudu (näiteks käive, kasumimarginaal, jne) kui ka andmeteaduslike mõistete abil (näiteks ennustustäpsus). Esialgse plaani paika panemine. 

<br><br>

<img src="https://i.pinimg.com/originals/4f/21/1a/4f211a6bf2d01e38094af9963663872d.jpg" height="400px" width="400px"/>

Metallitööstuse tehases kasutatakse suurte metallblokkide vormimiseks hüdraulilist pressi. Hüdraulilise pressi töötamise seisukoha pealt on oluline, et korras oleks järgmised komponendid: <br>
1) Jahuti <br>
2) Kraan <br>
3) Pump <br>
4) Akumulaator <br><br>

Aegajalt tuleb ette, et mõne sellise komponendi katki minemine või riknemine põhjustab hüdraulilisele pressile kahjusid, mille parandamine läheb maksma ca 5000 EUR. <br><br>

Komponentide täieliku hävimise vältimiseks õpetati välja tehnik, kes kontrollib iga hüdraulilise pressi töötsükli järel nende nelja komponendi tehnilist seisukorda. Kui mõne komponendi seisukord on halb, siis vahetatakse see komponent välja ning välditakse komponendi purunemisel tekkivad masina lisakahjustusi. <br><br>

Komponentide tehnilise sisukorra hindamine võtab iga tsükli järel aega ca 1 tund, ning ettevõte maksab tehnikule iga tunni eest 10 EUR.





## 2) Andmete mõistmine
See etapp algab andmete kogumisega ja sisaldab tegevusi
andmete hulga, hoiustamise viisi ning struktuuri tuvastamiseks. Lisaks tuvastatakse võimalikud andmekvaliteedi probleemid. <br><br>

Masinõppe ja statistilised mudelid üldiselt põhinevad andmete analüüsil. Enne probleemi uurimist või mudeli ehitamist on meil vaja andmeid, millele vajaminev mudel ehitada. Andmete kogumisel tasub silmas pidada järgnevaid asjaolusid: <br>
1) Kui keeruline või kulukas vajaminevaid andmeid on koguda, kas vaja on masinaid eelnevalt varustada mingite sensoritega? <br>
2) Kas juba leidub varasemalt kogutud andmeid, mida antud ülesande lahendamiseks saaks kasutada? <br>
3) Andmeid leidub tihti ettevõtte andmebaasidest, Exceli failidest ja ERP süsteemidest.

<br><br>
Andmestik on saadud eksperimentaalselt, mõttes sensorite abil hüdraulilise pressi erinevaid näitusid. Süsteem töötas 60-sekundiliste tsüklitena, mille käigus mõõtsid sensorid pressi rõhkusid, mahuvoolusid, temperatuuri ja hüdraulilise pressi nelja komponendi seisukorda: jahuti, kraan, pump, akkumulaator. Iga sensori mõõtmised on salvestatud eraldi faili. 

1. Rõhusensorid (PS1-6): 100 Hz (6 sensorit)
2. Mootori võimsuse sensor (EPS1): 100 Hz (1 sensor)
3. Mahuvoolu sensorid (FS1/2): 10 Hz (2 sensorit)
4. Temperatuuri sensorid (TS1-4): 1 Hz (4 sensorit)
5. Vibratsiooni sensor (VS1): 1 Hz (1 sensor)
6. Efektiivsustegur (SE): 1 Hz (1 sensor)
7. Virtuaalne jahutusefektviisuse sensor (CE): 1 Hz (1 sensor)
8. Virtuaalne jahutusvõimsuse sensor (CP): 1 Hz (1 sensor)

Hüdraulilise protsessi komponentide kulumised ja kvaliteedikaod on välja toodud profile.txt failis. Seal failis on kirjeldatud nelja komponendi kvaliteedid:

   1. Jahuti seisukord / %:
      3: peaaegu katki
      20: vähenenud efektviisus
      100: täielik efektviisus

   2. Kraani seisukord / %:
      100: optimaalne lülitus
      90: väike hilinevus
      80: suur hilinevus
      73: peaagu katki

   3. Sisemise pumba lekked:
      0: lekked puuduvad
      1: vähene lekkimine
      2: suured lekked

   4. Hüdrauliline akkumulaator / bar:
      130: optimaalne rõhk
      115: vähesel määral langenud rõhk
      100: suurel määral langenud rõhk
      90: peaaegu katki

## 3) Andmete ettevalmistamine
Kasulike andmete välja valimine, andmete puhastamine, andmetabelite loomine ja ühendamine. Selle etapi lõpuks tuleks andmed viia sellisele kujule, mida on sobiv ette sööta tehisintellekti meetoditele ja algoritmidele.

In [ ]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 2.1 MB/s 
     |████████████████████████████████| 192.9 MB 70 kB/s 
     |████████████████████████████████| 160 kB 21.5 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=1c2da4358e8f46f8187d70a3f28f9b83b76bf9450204c460821d8cd38dc9cc0e
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import cross_validate,GridSearchCV,RandomizedSearchCV
from tpot import TPOTClassifier
from scipy.stats import randint
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
!wget https://drive.google.com/uc?export=download&id=1EaVlcWDROQ69WTKoleGyIQrDxZy6fBlu

--2022-06-13 08:55:19--  https://drive.google.com/uc?export=download
Resolving drive.google.com (drive.google.com)... 142.250.31.113, 142.250.31.100, 142.250.31.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.31.113|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2022-06-13 08:55:19 ERROR 400: Bad Request.



In [ ]:
!jar xvf hydraulicsystem.zip

 inflated: CE.txt
 inflated: CP.txt
 inflated: EPS1.txt
 inflated: FS1.txt
 inflated: FS2.txt
 inflated: PS1.txt
 inflated: PS2.txt
 inflated: PS3.txt
 inflated: PS4.txt
 inflated: PS5.txt
 inflated: PS6.txt
 inflated: SE.txt
 inflated: TS1.txt
 inflated: TS2.txt
 inflated: TS3.txt
 inflated: TS4.txt
 inflated: VS1.txt
 inflated: description.txt
 inflated: documentation.txt
 inflated: profile.txt


In [ ]:
ps1 = np.genfromtxt(r"PS1.txt") # Rõhusensor 1
ps2 = np.genfromtxt(r"PS2.txt") # Rõhusensor 2
ps3 = np.genfromtxt(r"PS3.txt") # Rõhusensor 3
ps4 = np.genfromtxt(r"PS4.txt") # Rõhusensor 4
ps5 = np.genfromtxt(r"PS5.txt") # Rõhusensor 5
ps6 = np.genfromtxt(r"PS6.txt") # Rõhusensor 6

eps1 = np.genfromtxt(r"EPS1.txt") # Mootori võimsuse sensor

fs1 = np.genfromtxt(r"FS1.txt") # Mahusensor 1
fs2 = np.genfromtxt(r"FS2.txt") # Mahusensor 2

ts1 = np.genfromtxt(r"TS1.txt") # Temperatuuri sensor 1
ts2 = np.genfromtxt(r"TS2.txt") # Temperatuuri sensor 2
ts3 = np.genfromtxt(r"TS3.txt") # Temperatuuri sensor 3
ts4 = np.genfromtxt(r"TS4.txt") # Temperatuuri sensor 4

vs1 = np.genfromtxt(r"VS1.txt") # Vibratsiooni sensor 
se = np.genfromtxt(r"SE.txt") # Efektsiivuse sensor 
ce = np.genfromtxt(r"CE.txt") # Jahutusefektiivsuse sensor 
cp = np.genfromtxt(r"CP.txt") # Jahutusvõimsuse sensor 

Kuna ühe tsükli jooksul salvestatakse sensori väärtus 60+ korda, kuid komponentide kulumisi salvestatakse vaid ühe korra kogu tsükli jooksu, siis on meil mõistlik ka sensori andmeid ühe tsükli jaoks kirjeldada ühe arvuga. Üheks mõistlikuks valikuks on võtta iga tsükli sensornäitude aritmeetiline keskmine.

In [ ]:
ps1_arr=ps1.mean(axis=1)
ps2_arr=ps2.mean(axis=1)
ps3_arr=ps3.mean(axis=1)
ps4_arr=ps4.mean(axis=1)
ps5_arr=ps5.mean(axis=1)
ps6_arr=ps6.mean(axis=1)

eps1_arr=eps1.mean(axis=1)

fs1_arr=fs1.mean(axis=1)
fs2_arr=fs2.mean(axis=1)

ts1_arr=ts1.mean(axis=1)
ts2_arr=ts2.mean(axis=1)
ts3_arr=ts3.mean(axis=1)
ts4_arr=ts4.mean(axis=1)

vs1_arr=vs1.mean(axis=1)
se_arr=se.mean(axis=1)
ce_arr=ce.mean(axis=1)
cp_arr=cp.mean(axis=1)

In [ ]:
df=pd.DataFrame({"PS1":ps1_arr,"PS2":ps2_arr,"PS3":ps3_arr,"PS4":ps4_arr,"PS5":ps5_arr,"PS6":ps6_arr,
                 "EPS1":eps1_arr,"FS1":fs1_arr,"FS2":fs2_arr,"TS1":ts1_arr,"TS2":ts2_arr,"TS3":ts3_arr,"TS4":ts4_arr,
                 "VS1":vs1_arr,"SE":se_arr,"CE":ce_arr,"CP":cp_arr})

In [ ]:
df.head(10)

,PS1,PS2,PS3,PS4,PS5,PS6,EPS1,FS1,FS2,TS1,TS2,TS3,TS4,VS1,SE,CE,CP
0,160.673492,109.466914,1.991475,0.0,9.842169,9.728098,2538.929167,6.709815,10.304592,35.621983,40.978767,38.471017,31.745250,0.576950,59.157183,39.601350,1.862750
1,160.603320,109.354890,1.976234,0.0,9.635142,9.529488,2531.498900,6.715315,10.403098,36.676967,41.532767,38.978967,34.493867,0.565850,59.335617,25.786433,1.255550
2,160.347720,109.158845,1.972224,0.0,9.530548,9.427949,2519.928000,6.718522,10.366250,37.880800,42.442450,39.631950,35.646150,0.576533,59.543150,22.218233,1.113217
3,160.188088,109.064807,1.946576,0.0,9.438827,9.337429,2511.541633,6.720565,10.302678,38.879050,43.403983,40.403383,36.579467,0.569267,59.794900,20.459817,1.062150
4,160.000472,108.931434,1.922707,0.0,9.358762,9.260636,2503.449500,6.690308,10.237750,39.803917,44.332750,41.310550,37.427900,0.577367,59.455267,19.787017,1.070467
5,159.920210,108.887682,1.913285,0.0,9.301160,9.206877,2501.007067,6.699023,10.178720,40.659450,45.170617,42.124117,38.212067,0.572683,59.563333,19.149683,1.072083
6,159.672675,108.676466,1.888100,0.0,9.233942,9.143320,2494.416900,6.698573,10.140810,41.463633,45.947233,42.888117,38.932100,0.573033,59.789900,18.666383,1.081683
7,159.614452,108.651745,1.874894,0.0,9.194159,9.105058,2489.421533,6.678027,10.095978,42.215267,46.665233,43.519883,39.558967,0.572000,59.590617,18.178433,1.078700
8,159.475745,108.529738,1.858120,0.0,9.144616,9.057067,2484.419067,6.671652,10.039710,42.891983,47.339433,44.201250,40.080533,0.567067,59.608883,18.334867,1.115083
9,159.437997,108.510885,1.841063,0.0,9.104831,9.019265,2480.434867,6.659990,9.997762,43.532833,47.981417,44.797133,40.612550,0.571683,59.473733,18.205733,1.129133


In [ ]:
target = np.genfromtxt(r"profile.txt") # Jahutusvõimsuse sensor 

df_temp=pd.DataFrame(target,columns=["Cooler_Condition","Valve_Condition","Internal_Pump_Leakage","Hydraulic_Accumulator","Stable_Flag"])
df_temp.head()

,Cooler_Condition,Valve_Condition,Internal_Pump_Leakage,Hydraulic_Accumulator,Stable_Flag
0,3.0,100.0,0.0,130.0,1.0
1,3.0,100.0,0.0,130.0,1.0
2,3.0,100.0,0.0,130.0,1.0
3,3.0,100.0,0.0,130.0,1.0
4,3.0,100.0,0.0,130.0,1.0


2205

In [ ]:
df_final=pd.concat([df,df_temp],axis=1)

In [ ]:
df_final.head(20)

,PS1,PS2,PS3,PS4,PS5,PS6,EPS1,FS1,FS2,TS1,...,TS4,VS1,SE,CE,CP,Cooler_Condition,Valve_Condition,Internal_Pump_Leakage,Hydraulic_Accumulator,Stable_Flag
0,160.673492,109.466914,1.991475,0.000000e+00,9.842169,9.728098,2538.929167,6.709815,10.304592,35.621983,...,31.745250,0.576950,59.157183,39.601350,1.862750,3.0,100.0,0.0,130.0,1.0
1,160.603320,109.354890,1.976234,0.000000e+00,9.635142,9.529488,2531.498900,6.715315,10.403098,36.676967,...,34.493867,0.565850,59.335617,25.786433,1.255550,3.0,100.0,0.0,130.0,1.0
2,160.347720,109.158845,1.972224,0.000000e+00,9.530548,9.427949,2519.928000,6.718522,10.366250,37.880800,...,35.646150,0.576533,59.543150,22.218233,1.113217,3.0,100.0,0.0,130.0,1.0
3,160.188088,109.064807,1.946576,0.000000e+00,9.438827,9.337429,2511.541633,6.720565,10.302678,38.879050,...,36.579467,0.569267,59.794900,20.459817,1.062150,3.0,100.0,0.0,130.0,1.0
4,160.000472,108.931434,1.922707,0.000000e+00,9.358762,9.260636,2503.449500,6.690308,10.237750,39.803917,...,37.427900,0.577367,59.455267,19.787017,1.070467,3.0,100.0,0.0,130.0,1.0
5,159.920210,108.887682,1.913285,0.000000e+00,9.301160,9.206877,2501.007067,6.699023,10.178720,40.659450,...,38.212067,0.572683,59.563333,19.149683,1.072083,3.0,100.0,0.0,130.0,1.0
6,159.672675,108.676466,1.888100,0.000000e+00,9.233942,9.143320,2494.416900,6.698573,10.140810,41.463633,...,38.932100,0.573033,59.789900,18.666383,1.081683,3.0,100.0,0.0,130.0,1.0
7,159.614452,108.651745,1.874894,0.000000e+00,9.194159,9.105058,2489.421533,6.678027,10.095978,42.215267,...,39.558967,0.572000,59.590617,18.178433,1.078700,3.0,100.0,0.0,130.0,1.0
8,159.475745,108.529738,1.858120,0.000000e+00,9.144616,9.057067,2484.419067,6.671652,10.039710,42.891983,...,40.080533,0.567067,59.608883,18.334867,1.115083,3.0,100.0,0.0,130.0,1.0
9,159.437997,108.510885,1.841063,0.000000e+00,9.104831,9.019265,2480.434867,6.659990,9.997762,43.532833,...,40.612550,0.571683,59.473733,18.205733,1.129133,3.0,100.0,0.0,130.0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

X = df_final.drop(["Cooler_Condition","Valve_Condition","Internal_Pump_Leakage","Hydraulic_Accumulator","Stable_Flag"], axis=1)
y = df_final["Valve_Condition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## 4) Mudeldamine
Sobivate meetodite ja tehnikate valimine ning nende rakendamine
loodud andmetele. Lisaks hinnatakse siin mudelite sobivust matemaatiliselt, vastavalt andmeteaduslikele meetoditele nagu täpsus, jne.



Arvulistele andmetele ei pea ilmtingimata rakendama ainult regressioonmudeleid. Näiteks antud juhul on meil uuritavateks tunnusteks küll arvud, aga neid erinevaid arve on nii vähe (kõigest 4 erinevat), mistõttu saab selliste väärtuste ennustamiseks kasutada ka klassifitseerimismudeleid.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def trainClassifiers():

  lr = LogisticRegression()
  knn = KNeighborsClassifier(3)
  dt = DecisionTreeClassifier()
  svm = SVC()
  rf = RandomForestClassifier()

  #lr.fit(X_train,y_train)
  #knn.fit(X_train,y_train)
  #dt.fit(X_train,y_train)
  #svm.fit(X_train,y_train)
  #rf.fit(X_train,y_train)

  #y_pred_lr = lr.predict(X_test)
  #y_pred_knn = knn.predict(X_test)
  #y_pred_dt = dt.predict(X_test)
  #y_pred_svm = svm.predict(X_test)
  #y_pred_rf = rf.predict(X_test)

  #acc_lr = accuracy_score(y_pred_lr, y_test)
  #acc_knn = accuracy_score(y_pred_knn, y_test)
  #acc_dt = accuracy_score(y_pred_dt, y_test)
  #acc_svm = accuracy_score(y_pred_svm, y_test)
  #acc_rf = accuracy_score(y_pred_rf, y_test)

  acc_lr = cross_val_score(lr, X, y, cv=5).mean()
  acc_knn = cross_val_score(knn, X, y, cv=5).mean()
  acc_dt = cross_val_score(dt, X, y, cv=5).mean()
  acc_svm = cross_val_score(svm, X, y, cv=5).mean()
  acc_rf = cross_val_score(rf, X, y, cv=5).mean()

  print("Accuracy of LogisticRegression:", acc_lr)
  print("Accuracy of KNeighborsClassifier:", acc_knn)
  print("Accuracy of DecisionTreeClassifier:", acc_dt)
  print("Accuracy of SupportVectorMachine:", acc_svm)
  print("Accuracy of RandomForestClassifier:", acc_rf)

In [ ]:
print("TRAINING MODELS FOR TARGET: Cooler_Condition")
y = df_final["Cooler_Condition"]
trainClassifiers()

print("TRAINING MODELS FOR TARGET: Valve_Condition")
y = df_final["Valve_Condition"]
trainClassifiers()

print("TRAINING MODELS FOR TARGET: Internal_Pump_Leakage")
y = df_final["Internal_Pump_Leakage"]
trainClassifiers()

print("TRAINING MODELS FOR TARGET: Hydraulic_Accumulator")
y = df_final["Hydraulic_Accumulator"]
trainClassifiers()

TRAINING MODELS FOR TARGET: Cooler_Condition
Accuracy of LogisticRegression: 0.9800453514739228
Accuracy of KNeighborsClassifier: 0.9764172335600907
Accuracy of DecisionTreeClassifier: 0.981859410430839
Accuracy of SupportVectorMachine: 0.5931972789115646
Accuracy of RandomForestClassifier: 0.9800453514739228
TRAINING MODELS FOR TARGET: Valve_Condition
Accuracy of LogisticRegression: 0.4811791383219955
Accuracy of KNeighborsClassifier: 0.4217687074829932
Accuracy of DecisionTreeClassifier: 0.5755102040816327
Accuracy of SupportVectorMachine: 0.5102040816326531
Accuracy of RandomForestClassifier: 0.582312925170068
TRAINING MODELS FOR TARGET: Internal_Pump_Leakage
Accuracy of LogisticRegression: 0.6911564625850339
Accuracy of KNeighborsClassifier: 0.7714285714285714
Accuracy of DecisionTreeClassifier: 0.8721088435374149
Accuracy of SupportVectorMachine: 0.5537414965986394
Accuracy of RandomForestClassifier: 0.8970521541950113
TRAINING MODELS FOR TARGET: Hydraulic_Accumulator
Accuracy of 

In [ ]:
print(df_final["Cooler_Condition"].nunique())
print(df_final["Valve_Condition"].nunique())
print(df_final["Internal_Pump_Leakage"].nunique())
print(df_final["Hydraulic_Accumulator"].nunique())

3
4
3
4


## 5) Projekti tulemuste hindamine
Selleks etapiks on juba andmeanalüütiliste
kriteeriumite järgi sobivad mudelid valitud ja loodud, ning nüüd hinnatakse nende tulemusi vastavalt ärilistele parameetritele. Kontrollitakse üle kõik eelnevalt tehtud sammud ning hinnatakse kas kõik on püstitatud ärilistele eesmärkidele vastavuses. Selle etapi lõpuks valitakse töötav lahendus, mida saab eesmärgipäraselt kasutada.

## 7) Kasutusele võtmine ehk juurutamine
Saadud teadmiste rakendamine otsuste langetamisel igapäevatöös, plaani loomine tulemuste jälgimiseks ja lahenduse hooldamiseks tulevikus. Sõltuvalt eesmärgist võivad tulemused olla väga erinevad, alustades lihtsast analüüsi raportist ning lõpetades keerukate masinõppe rakenduste püsti panekuga.